In [ ]:
# A notebook for checking the latest sentinel images for a defined area.
# First, need a means of storing previous imagery.

In [1]:
# These lines require user intervention
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/drive')

In [4]:
from google.oauth2 import service_account
from google.cloud import storage
import ee

# Skip earth engine login clickety click
credentials = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/earth-engine-workflow-013a80bf4f5b.json')
service_account = 'harvester@earth-engine-workflow.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '/content/drive/MyDrive/earth-engine-workflow-013a80bf4f5b.json')
ee.Initialize(credentials)

# Check for bucket and create if not present
client = storage.Client(credentials=credentials)
client
OUTPUT_BUCKET = 'airsignal2023'

def create_bucket(bucket_name):
    storage_client = storage.Client()    
    bucket = storage_client.create_bucket(bucket_name)
    print('Bucket {} created'.format(bucket.name))

def bucket_exists(bucket_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    try:
        bucket.reload()
        return True
    except Exception as e:
        print(e)
        return False

# Replace <BUCKET_NAME> with the name of your bucket
if bucket_exists(OUTPUT_BUCKET):
  print('Bucket exists')
else:
  print('Bucket does not exist')
  create_bucket(OUTPUT_BUCKET)


Bucket exists


In [ ]:
#!pip install earthengine-api

In [5]:
# Anglesea Heath
xmin = 144.3
xmax = 144.25
ymin = -38.3
ymax = -38.44

ROI_POLY = ee.Geometry.Polygon([[[xmin, ymax],
                          [xmin, ymin],
                          [xmax, ymin],
                          [xmax, ymax]]])
lat,lon = (ymin+ymax)/2,(xmin+xmax)/2

In [6]:
# Define a first set of dates
START_DATE = "2023-01-01"
END_DATE =  "2023-02-01"

In [14]:
# SENTINEL FUNCTIONS
# Use QA for mask then discard
#https://github.com/davemlz/ee-catalog-scale-offset-params/blob/main/list/ee-catalog-scale-offset-parameters.json
#bands_10m =
#bands_60m = 
#SCALE_BANDS =
BANDS = ['B2','B3','B4','B8','QA60']
CLOUD_LIMIT = 50
def se2mask(image):
    quality_band = image.select('QA60')
    # using the bit mask for clouds and cirrus clouds respectively
    cloudmask = 1 << 10
    cirrusmask = 1 << 11    
    # we only want clear skies
    mask = quality_band.bitwiseAnd(cloudmask).eq(0) and (quality_band.bitwiseAnd(cirrusmask).eq(0)) 
    # we'll divide by 10000 to make interpreting the reflectance values easier
    return image.updateMask(mask)#.divide(10000)

def scale(image):
  bands = ['B2','B3','B4','B8']
  image = image.select(bands).multiply(0.0001)
  #image = image.addBands(im2.select(bands))
  return image

In [46]:
imcol = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(ROI_POLY).filterDate(
    START_DATE, END_DATE).filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",CLOUD_LIMIT)).select(BANDS).map(scale)
#.map(se2mask)
# map(lambda img: img.divide(10000))
# NB this can be expensive to run!
#print(ee.Number.getInfo(imcol.size()))

In [41]:
# Some information about the image list we are processing
def get_name(img):
  return(ee.Image(img).getString('id'))

imlist = imcol.toList(imcol.size())
#unique_dates = imlist.map(lambda im: ee.Image(im).date().format("YYYY-MM-dd")) 
#print(ee.List.getInfo(unique_dates))
unique_ids = imlist.map(lambda im: ee.Image(im).id()  )
idslist = ee.List.getInfo(unique_ids)

In [192]:
PROJECT_ID = "earth-engine-workflow"
BUCKET_NAME = "airsignal2023"
REGION = "us-central1"
USER_NAME = 'charlesan'

In [22]:
import json
json.dumps(idslist)

'["20230105T002049_20230105T002432_T54HYC", "20230105T002049_20230105T002432_T55HBT", "20230110T002051_20230110T002415_T54HYC", "20230110T002051_20230110T002415_T55HBT", "20230115T002049_20230115T002358_T54HYC", "20230115T002049_20230115T002358_T55HBT", "20230125T002049_20230125T002320_T54HYC", "20230125T002049_20230125T002320_T55HBT"]'

In [24]:
# Upload list of files we processed
client = storage.Client(credentials=credentials)
BUCKET = client.get_bucket(OUTPUT_BUCKET)
filename = "idlist.json"
blob = BUCKET.blob(filename)
json_object = idslist

blob.upload_from_string(
    data=json.dumps(json_object),
    content_type='application/json'
)
result = filename + ' upload complete'

In [16]:
img = imcol.first().clip(ROI_POLY)

In [22]:
projection = img.select('B2').projection().getInfo();

In [17]:
ee.List.getInfo(img.bandNames())

['B2', 'B3', 'B4', 'B8']

In [24]:
projection['crs']

'EPSG:32754'

In [26]:
# Save 10m bands
task = ee.batch.Export.image.toCloudStorage(**{
  'image': img,
  'description': 'image_export_job',
  'crs': 'EPSG:7854',
  'bucket': OUTPUT_BUCKET,
  'fileNamePrefix': 'angle',
  #'dimensions':,
  'crs': projection['crs'],
  'scale':10,
  'crsTransform': projection['transform'],
  'region': ROI_POLY,
  'fileFormat': 'GeoTIFF',
  'formatOptions': {
    'cloudOptimized': True
  },
  'maxPixels': 1e8})
task.start()

In [27]:
blobs = client.list_blobs(OUTPUT_BUCKET)
# Note: The call returns a response only when the iterator is consumed.
for blob in blobs:
  print(blob.name)

idlist.json


In [29]:
print(ee.batch.Task.list())

[<Task MYTFJ7KAE2KTNHG54PEDIKQZ EXPORT_IMAGE: image_export_job (COMPLETED)>]


In [170]:
names = img.bandNames()
ee.List.getInfo(names)

In [48]:
#range = imcol.reduceColumns(ee.Reducer.minMax(), ['system:time_start'])
#ee.Dictionary.getInfo(range)

# This function adds a band representing the image timestamp.
def addTime(image):
  return image.addBands(image.getNumber('system:time_start'))
# Map the function over the collection and display the result.
imcol = imcol.map(addTime)

In [ ]:
# why this no work?
unique_dates = imlist.map(lambda im: ee.Image(im).date().format("YYYY-MM-dd")) 
print(ee.List.getInfo(unique_dates))

In [ ]:
# broke
#def getDate(image):
#  return image.getNumber('system:time_start')
#dates = imlist.map(getDate)

In [51]:
dates

In [34]:
def mosaicByDate(imcol):
  imlist = imcol.toList(imcol.size())
  unique_dates = imlist.map(lambda im: ee.Image(im).date().format("YYYY-MM-dd")).distinct()

  def match_dates(d):
    d = ee.Date(d)
    dateString = ee.Date(d).format('yyyy-MM-dd')
    im = imcol.filterDate(d, d.advance(1, "day")).mosaic()
    return im.set(
        "system:time_start", d.millis(), 
        "system:id", d.format("YYYY-MM-dd")).rename(dateString)

  mosaic_imlist = unique_dates.map(match_dates)

  return ee.ImageCollection(mosaic_imlist)

moscol = mosaicByDate(imcol)

In [110]:
moscol

In [ ]:
imlist = imcol.toList(imcol.size())
dates = imlist.map(lambda im: ee.Image(im).date()  )
idslist = ee.List.getInfo(dates)
#print(ee.List.getInfo(idslist))

In [109]:
unique_dates

In [ ]:
# Save this image collection

In [ ]:
# Define another date
# This time search the local archives to see if these dates are present

In [1]:
%%capture
#!pip install geemap

In [3]:
#import numpy as np
#import geemap
#import pandas as pd
#import plotnine
#from scipy import stats
#import tensorflow as tf
#print(tf.__version__)
#print(folium.__version__)

In [ ]:
import geemap

In [ ]:
rgb = ['B4','B3','B2','B8']
rgbViz = {"min":0.0, "max":0.3,"bands":rgb}
ndviViz = {"min":0.0, "max":1.0,"bands":['NDVI']}

In [ ]:
"""COPERNICUS/S2_SR_HARMONIZED": {
        "AOT": {
            "offset": 0.0,
            "scale": 0.001
        },
        "B1": {
            "offset": 0.0,
            "scale": 0.0001
        },
        "B11": {
            "offset": 0.0,
            "scale": 0.0001
        },
        "B12": {
            "offset": 0.0,
            "scale": 0.0001
        },
        "B2": {
            "offset": 0.0,
            "scale": 0.0001
        },
        "B3": {
            "offset": 0.0,
            "scale": 0.0001
        },
        "B4": {
            "offset": 0.0,
            "scale": 0.0001
        },
        "B5": {
            "offset": 0.0,
            "scale": 0.0001
        },
        "B6": {
            "offset": 0.0,
            "scale": 0.0001
        },
        "B7": {
            "offset": 0.0,
            "scale": 0.0001
        },
        "B8": {
            "offset": 0.0,
            "scale": 0.0001
        },
        "B8A": {
            "offset": 0.0,
            "scale": 0.0001
        },
        "B9": {
            "offset": 0.0,
            "scale": 0.0001
        },
        "MSK_CLDPRB": {
            "offset": 0.0,
            "scale": 1.0
        },
        "MSK_SNWPRB": {
            "offset": 0.0,
            "scale": 1.0
        },
        "QA10": {
            "offset": 0.0,
            "scale": 1.0
        },
        "QA20": {
            "offset": 0.0,
            "scale": 1.0
        },
        "QA60": {
            "offset": 0.0,
            "scale": 1.0
        },
        "SCL": {
            "offset": 0.0,
            "scale": 1.0
        },
        "TCI_B": {
            "offset": 0.0,
            "scale": 1.0
        },
        "TCI_G": {
            "offset": 0.0,
            "scale": 1.0
        },
        "TCI_R": {
            "offset": 0.0,
            "scale": 1.0
        },
        "WVP": {
            "offset": 0.0,
            "scale": 0.001
        }
    },"""